In [1]:
import subprocess
import pandas as pd
import os
import sys
import pprint
import local_models.local_models
import logging
import ml_battery.log

logger = logging.getLogger(__name__)

In [2]:
data_dir = "/home/brown/disk2/eeg/Phasespace/Phasespace/data/eeg-text" 
transformed_data_dir = "/home/brown/disk2/eeg/transformed_data"

In [3]:
data_info = pd.read_csv(os.path.join(data_dir, "fileinformation.csv"), skiprows=1).iloc[:,2:]

In [7]:
data_info.iloc[39]

Unnamed: 2           DAT.F00265
Number of Records       6092424
Time of Seizure           24120
Unnamed: 5                  NaN
Point of Seizure        6030000
250                     6030000
Name: 39, dtype: object

In [4]:
data_info.shape
how_many_epis = len([which for which in range(data_info.shape[0]) if data_info.iloc[which,4]>0])
how_many_epis

40

In [ ]:
timelog = local_models.local_models.loggin.TimeLogger(
    logger=logger, 
    how_often=1, total=how_many_epis, 
    tag="transforming_eeg_gprs")
for i in range(39,data_info.shape[0]):
    data_file = data_info.iloc[i,0]
    data_epipoint = data_info.iloc[i,4]
    data_len = data_info.iloc[i,1]
    if data_len > data_epipoint > 0:
        with timelog:
            transformed_data_file_dir = os.path.join(transformed_data_dir, data_file)
            if not os.path.isdir(transformed_data_file_dir):
                result = subprocess.run(
                    ["python3", "Todd_eeg_utils.py", data_file, str(data_epipoint)],
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE
                )
                print(result.stderr.decode("utf-8"))